In [90]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast

from pymongo import ReplaceOne
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [91]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]

stagingPlaatje = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
stagingDigiFoto = stagingDb[config.COLL_STAGING_DIGIFOTOS]
collection = analyseDb[config.COLL_ANALYSE]
cleancollection = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']
AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"

In [92]:
df_foto = pd.DataFrame(list(collection.find({"soort": "Foto"})))
df_foto

,_id,fileName,imageUUID,imageMiddleUUID,imageThumbUUID,fileType,directory,mime_type,fototype,soort,...,key_project,key_project_type,key_vondst,artefactnr,key_artefact,fotosubnr,aantal,gewicht,materiaalcode,materiaalspecifiek
0,62222c7328d9b2f1d21d2b45,_DSC2957.jpg,62222c7328d9b2f1d21d2b3f,62222c7328d9b2f1d21d2b41,62222c7328d9b2f1d21d2b43,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62222c7428d9b2f1d21d2b4c,_DSC2958.jpg,62222c7328d9b2f1d21d2b46,62222c7428d9b2f1d21d2b48,62222c7428d9b2f1d21d2b4a,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62222c7428d9b2f1d21d2b53,_DSC2959.jpg,62222c7428d9b2f1d21d2b4d,62222c7428d9b2f1d21d2b4f,62222c7428d9b2f1d21d2b51,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62222c7528d9b2f1d21d2b5a,_DSC2960.jpg,62222c7428d9b2f1d21d2b54,62222c7528d9b2f1d21d2b56,62222c7528d9b2f1d21d2b58,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62222c7528d9b2f1d21d2b61,_DSC2961.jpg,62222c7528d9b2f1d21d2b5b,62222c7528d9b2f1d21d2b5d,62222c7528d9b2f1d21d2b5f,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294,62222dc9e87e32e116231216,DC121_G0029.jpg,62222dc9e87e32e11623120f,62222dc9e87e32e116231212,62222dc9e87e32e116231214,.jpg,/input/projectdatabase/digidepot_DC/DC198_Burg...,image/jpeg,G,Foto,...,PDC121,GPDC121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4295,62222dcae87e32e11623121f,DC121_G0030.jpg,62222dc9e87e32e116231217,62222dcae87e32e11623121b,62222dcae87e32e11623121d,.jpg,/input/projectdatabase/digidepot_DC/DC198_Burg...,image/jpeg,G,Foto,...,PDC121,GPDC121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4296,62222dcae87e32e116231228,DC121_G0031.jpg,62222dcae87e32e116231220,62222dcae87e32e116231224,62222dcae87e32e116231226,.jpg,/input/projectdatabase/digidepot_DC/DC198_Burg...,image/jpeg,G,Foto,...,PDC121,GPDC121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4297,62222dcae87e32e116231232,DC121_G0032.jpg,62222dcae87e32e116231229,62222dcae87e32e11623122e,62222dcae87e32e116231230,.jpg,/input/projectdatabase/digidepot_DC/DC198_Burg...,image/jpeg,G,Foto,...,PDC121,GPDC121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df_fotobeschr = pd.DataFrame(list(collection.find({"soort": "Fotobeschrijving"})))
df_fotobeschr["abcd-nr"] = df_fotobeschr.apply(lambda x: x["pad"].split('\\')[-1].lower() if x["pad"] and type(x["pad"]) == str else '', axis=1) 
df_fotobeschr

,_id,brondata,projectcd,pad,soort,key,key_project,putnr,spoornr,profiel,vondstnr,vondstkey_met_putnr,key_vondst,subnr,bestandsnaam,datum,omschrijving,vlaknr,richting,abcd-nr
0,62222c8585c209ee9414c7ab,"{'_id': 62222c8585c209ee9414c7ab, 'ID': 203, '...",DB034,#T:\ARCHEOLOGIE\DIA\A5001\A5703.JPG,Fotobeschrijving,PDB034,PDB034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a5703.jpg
1,62222c8585c209ee9414c7c5,"{'_id': 62222c8585c209ee9414c7c5, 'ID': 229, '...",DB034,#T:\ARCHEOLOGIE\DIA\A5001\A5729.JPG,Fotobeschrijving,PDB034,PDB034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a5729.jpg
2,62222c8585c209ee9414c7c6,"{'_id': 62222c8585c209ee9414c7c6, 'ID': 230, '...",DB034,#T:\ARCHEOLOGIE\DIA\A5001\A5730.JPG,Fotobeschrijving,PDB034,PDB034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a5730.jpg
3,62222c8585c209ee9414c7ec,"{'_id': 62222c8585c209ee9414c7ec, 'ID': 268, '...",DB034,#T:\ARCHEOLOGIE\DIA\A5001\A5768.JPG,Fotobeschrijving,PDB034,PDB034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a5768.jpg
4,62222c8585c209ee9414c7ed,"{'_id': 62222c8585c209ee9414c7ed, 'ID': 269, '...",DB034,#T:\ARCHEOLOGIE\DIA\A5001\A5769.JPG,Fotobeschrijving,PDB034,PDB034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a5769.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,62222cccc023e60924b60b19,"{'_id': 62222cccc023e60924b60b19, 'ID': 126, '...",HHD002,#T:\ARCHEOLOGIE\DIA\A6001\A6373.JPG,Fotobeschrijving,PHHD002,PHHD002,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a6373.jpg
2562,62222cccc023e60924b60b1a,"{'_id': 62222cccc023e60924b60b1a, 'ID': 127, '...",HHD002,#T:\ARCHEOLOGIE\DIA\A6001\A6374.JPG,Fotobeschrijving,PHHD002,PHHD002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a6374.jpg
2563,62222cccc023e60924b60b1b,"{'_id': 62222cccc023e60924b60b1b, 'ID': 128, '...",HHD002,#T:\ARCHEOLOGIE\DIA\A6001\A6375.JPG,Fotobeschrijving,PHHD002,PHHD002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a6375.jpg
2564,62222cccc023e60924b60b1c,"{'_id': 62222cccc023e60924b60b1c, 'ID': 129, '...",HHD002,#T:\ARCHEOLOGIE\DIA\A6001\A6376.JPG,Fotobeschrijving,PHHD002,PHHD002,NaN,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,a6376.jpg


In [94]:
df_fotokoppel = pd.DataFrame(list(collection.find({"soort": "Fotokoppel"})))
df_fotokoppel["abcd-nr"] = df_fotokoppel.apply(lambda x: x["abcd-nr"].lower() if x["abcd-nr"] and type(x["abcd-nr"]) == str else '', axis=1) 
df_fotokoppel

,_id,brondata,projectcd,abcd-nr,bestandsnaam,soort,key_bestandsnaam,materiaalgroep
0,62222c741307d4f73c75dab1,"{'_id': 62222c741307d4f73c75dab1, 'Uniek Numme...",DIGIFOTOS,b5250.jpg,Bruikleen_Tentoonstelling_01.jpg,Fotokoppel,BBruikleen_Tentoonstelling_01.jpg,NaN
1,62222c741307d4f73c75dab2,"{'_id': 62222c741307d4f73c75dab2, 'Uniek Numme...",DIGIFOTOS,b5251.jpg,Bruikleen_Tentoonstelling_02.jpg,Fotokoppel,BBruikleen_Tentoonstelling_02.jpg,NaN
2,62222c741307d4f73c75dab3,"{'_id': 62222c741307d4f73c75dab3, 'Uniek Numme...",DIGIFOTOS,b5252.jpg,Bruikleen_Tentoonstelling_03.jpg,Fotokoppel,BBruikleen_Tentoonstelling_03.jpg,NaN
3,62222c741307d4f73c75dab4,"{'_id': 62222c741307d4f73c75dab4, 'Uniek Numme...",DIGIFOTOS,b5253.jpg,Bruikleen_Tentoonstelling_04.jpg,Fotokoppel,BBruikleen_Tentoonstelling_04.jpg,NaN
4,62222c741307d4f73c75dab5,"{'_id': 62222c741307d4f73c75dab5, 'Uniek Numme...",DIGIFOTOS,b5254.jpg,Bruikleen_Tentoonstelling_05.jpg,Fotokoppel,BBruikleen_Tentoonstelling_05.jpg,NaN
...,...,...,...,...,...,...,...,...
31464,62222c791307d4f73c765599,"{'_id': 62222c791307d4f73c765599, 'Uniek Numme...",DIGIFOTOS,,MD22-06_P10_H0021_1_2.jpg,Fotokoppel,BMD22-06_P10_H0021_1_2.jpg,Aardewerk
31465,62222c791307d4f73c76559a,"{'_id': 62222c791307d4f73c76559a, 'Uniek Numme...",DIGIFOTOS,,MD22-06_P10_H0021_2_1.jpg,Fotokoppel,BMD22-06_P10_H0021_2_1.jpg,Aardewerk
31466,62222c791307d4f73c76559b,"{'_id': 62222c791307d4f73c76559b, 'Uniek Numme...",DIGIFOTOS,,MD22-06_P10_H0024_1_1.jpg,Fotokoppel,BMD22-06_P10_H0024_1_1.jpg,Aardewerk
31467,62222c791307d4f73c76559c,"{'_id': 62222c791307d4f73c76559c, 'Uniek Numme...",DIGIFOTOS,,MD22-06_P10_H0026_1_1.jpg,Fotokoppel,BMD22-06_P10_H0026_1_1.jpg,Aardewerk


In [95]:
regexTable = re.compile(r'\'table\': \'(.*?)\'') # regex to replace Object
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getTable(brondata):  
    try:
        return regexTable.search(str(brondata)).group(1)
    except:
        return None

def getProject(brondata):    
    try:
        return regexProject.search(str(brondata)).group(1)
    except:
        return None


df_merge = df_foto.merge(df_fotokoppel, how="left", right_on="bestandsnaam", left_on="fileName", suffixes=("", "_koppel"))
df_merge = df_merge.merge(df_fotobeschr, how="left", on=["abcd-nr", "projectcd"], suffixes=("", "_beschr"))
df_merge['soort'] = 'Foto'
df_merge['brondata'] = df_merge.apply(lambda x: [x['brondata'], x['brondata_beschr']], axis=1)
df_merge['wasstraat'] = df_merge.apply(lambda x: {"projects": [getProject(elem) for elem in x['brondata'] if getProject(elem)], "tables": [getTable(elem) for elem in x['brondata'] if getTable(elem)]}, axis=1)  
df_merge = df_merge[['_id', 'fileName', 'imageUUID', 'imageMiddleUUID', 'imageThumbUUID',
       'fileType', 'directory', 'mime_type', 'fototype', 'soort', 'projectcd',
       'materiaal', 'putnr', 'vondstnr', 'fotonr', 'vondstkey_met_putnr',
       'key', 'key_project', 'key_project_type', 'key_vondst', 'artefactnr',
       'key_artefact', 'fotosubnr', 'aantal', 'gewicht', 'materiaalcode',
       'materiaalspecifiek', 'brondata', 'materiaalgroep',
       'pad', 'spoornr', 'profiel', 'subnr', 'datum', 'omschrijving', 'vlaknr', 'richting', 'wasstraat']]
df_merge.head(5)

,_id,fileName,imageUUID,imageMiddleUUID,imageThumbUUID,fileType,directory,mime_type,fototype,soort,...,materiaalgroep,pad,spoornr,profiel,subnr,datum,omschrijving,vlaknr,richting,wasstraat
0,62222c7328d9b2f1d21d2b45,_DSC2957.jpg,62222c7328d9b2f1d21d2b3f,62222c7328d9b2f1d21d2b41,62222c7328d9b2f1d21d2b43,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,"{'projects': [], 'tables': []}"
1,62222c7428d9b2f1d21d2b4c,_DSC2958.jpg,62222c7328d9b2f1d21d2b46,62222c7428d9b2f1d21d2b48,62222c7428d9b2f1d21d2b4a,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,"{'projects': [], 'tables': []}"
2,62222c7428d9b2f1d21d2b53,_DSC2959.jpg,62222c7428d9b2f1d21d2b4d,62222c7428d9b2f1d21d2b4f,62222c7428d9b2f1d21d2b51,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,"{'projects': [], 'tables': []}"
3,62222c7528d9b2f1d21d2b5a,_DSC2960.jpg,62222c7428d9b2f1d21d2b54,62222c7528d9b2f1d21d2b56,62222c7528d9b2f1d21d2b58,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,"{'projects': [], 'tables': []}"
4,62222c7528d9b2f1d21d2b61,_DSC2961.jpg,62222c7528d9b2f1d21d2b5b,62222c7528d9b2f1d21d2b5d,62222c7528d9b2f1d21d2b5f,.jpg,/input/fotos/DC97/L Fotos/H (Objectfoto's)/Metaal,image/jpeg,N,Foto,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,"{'projects': [], 'tables': []}"


In [96]:
updates=[ ReplaceOne({"_id": record['_id']}, record, upsert=True) for record in [v.dropna().to_dict() for k,v in df_merge.iterrows()]]  # 
result = cleancollection.bulk_write(updates)


In [105]:
def is_empty(item):
    if not item or item is None or pd.isna(item) or item == "":
        return True
    else: 
        return False

def not_empty(item):
    return not is_empty(item)

def firstValue(*kargs):
    for x in kargs:
        if not_empty(x):
            return x
    return None
    

In [109]:
firstValue(np.nan,"", 2,4)

2